In [8]:
%pip install scikit-learn ydata-profiling ipywidgets
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

Note: you may need to restart the kernel to use updated packages.


In [3]:
df = pd.read_csv('../../data/condensed_ajio.csv')
df.drop(['channel'], axis=1, inplace=True)
df = df.dropna(axis=0, subset=['product_id'])
df.drop(['product_id'], axis=1, inplace=True)
df.drop(['return_qty'], axis=1, inplace=True)
df.drop(['net_qty'], axis=1, inplace=True)
# # result = 1 year, 10 months, 5 days

# styles = pd.read_csv('../../data/styles.csv')
# sizes = pd.read_csv('../../data/sizes.csv')
# # delete one product_id col
# df = pd.merge(df, styles, on='product_id',how='left')
# df = pd.merge(df, sizes, on='ean',how='left')

In [19]:
# # filling missing values
# df['master_mrp'] = df['master_mrp'].fillna(df['master_mrp'].mode()[0])
# df['master_cost'] = df['master_cost'].fillna(df['master_cost'].median())
# df['avg_profit_per_pc'] = df['avg_profit'] / df['net_qty']
# df['avg_profit_per_pc'] = df['avg_profit_per_pc'].fillna(df['avg_profit_per_pc'].median())
# df['avg_profit'] = df['avg_profit_per_pc'] * df['net_qty']
profile = ProfileReport(df, title="Pandas Profiling Report",minimal=True)
profile.to_file("sales-combined.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

In [4]:
encoder = OneHotEncoder(sparse=False)
df_encoded = encoder.fit_transform(df[['ean']])
df_encoded = pd.DataFrame(df_encoded, columns=encoder.get_feature_names_out())

df.drop(['ean'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)
df = pd.concat([df, df_encoded], axis=1)
df

/Users/sud/opt/miniconda3/envs/crafted/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,date,sold_qty,avg_mrp,avg_discount,avg_sp,tigc_margin,avg_profit,master_mrp,master_cost,ean_8907955080434,...,ean_8907979111770,ean_8907979111787,ean_8907979111794,ean_8907979111800,ean_8907979111817,ean_8907979111824,ean_8907979111848,ean_8907979111855,ean_8907979111862,ean_8907979111879
0,2018-09-27,1,2499.0,0.000000,2499.00,1624.350,1108.350,2499.0,516.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-09-29,1,1899.0,54.976303,855.00,555.750,20.750,1899.0,535.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-09-30,1,1899.0,54.976303,855.00,555.750,20.750,1899.0,535.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-10-01,1,1899.0,54.976303,855.00,555.750,20.750,1899.0,535.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2018-10-01,1,1899.0,54.976303,855.00,555.750,20.750,1899.0,535.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70940,2020-07-31,0,1499.0,60.000000,599.60,389.740,77.740,1499.0,312.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70941,2020-07-31,0,1499.0,60.000000,599.60,389.740,16.740,1499.0,373.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70942,2020-07-31,1,1299.0,62.000000,493.62,320.853,63.853,1299.0,257.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70943,2020-07-31,1,1299.0,62.000000,493.62,320.853,63.853,1299.0,257.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df["date"] = pd.to_datetime(df["date"])
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["week"] = df["date"].dt.isocalendar().week
df.sort_values(by=['date'], inplace=True, ascending=True)

df_train = df[df['date']<'2020-03-20']
df_test = df[df['date']>='2020-03-20']
df.drop(['date'], axis=1, inplace=True)

col_to_predict = 'sold_qty'
all_cols = list(df.columns)
all_cols.remove(col_to_predict)
x_cols = all_cols 
y_col = [col_to_predict]

X_train = df_train[x_cols]
Y_train = df_train[y_col].values
X_test = df_test[x_cols]
Y_test = df_test[y_col].values

In [9]:
reg = LogisticRegression('elasticnet')
reg.fit(X_train, Y_train)

/Users/sud/opt/miniconda3/envs/crafted/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sud/opt/miniconda3/envs/crafted/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [10]:
# Create and fit the model
 # Instantiate the LinearRegression object
 # Train the model on the training set

# Make predictions on the test set
y_pred = reg.predict(X_test) # Predict the sales for the test set

# Evaluate the model performance
mse = mean_squared_error(Y_test, y_pred) # Calculate the mean squared error
rmse = np.sqrt(mse) # Calculate the root mean squared error
r2 = r2_score(Y_test, y_pred) # Calculate the R-squared score

# Print the results
print("MSE:", mse)
print("RMSE:", rmse)
print("R2:", r2)

MSE: 0.48100050530571
RMSE: 0.6935419996696018
R2: 0.6665926831337212


In [9]:
import matplotlib.pyplot as plt
plt.plot(X_test, y_pred, color="red", label="Predicted") # Plot the predicted sales as a red line
plt.xlabel("Date") # Add x-axis label
plt.ylabel("Sales") # Add y-axis label
plt.legend() # Add legend
plt.show() # Show the plot

/Users/sud/opt/miniconda3/envs/crafted/lib/python3.10/site-packages/IPython/core/pylabtools.py:152: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)


ValueError: Image size of 562x125739 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 640x480 with 1 Axes>

In [ ]:
# from sklearn.preprocessing import OneHotEncoder, StandardScaler

# # Encode the product name as dummy variables
# encoder = OneHotEncoder(sparse=False) # Instantiate the OneHotEncoder object
# X_encoded = encoder.fit_transform(X[["product_name"]]) # Fit and transform the product name column
# X_encoded = pd.DataFrame(X_encoded, columns=encoder.get_feature_names()) # Convert to a dataframe with column names

# # Normalize the EAN number to a standard scale
# scaler = StandardScaler() # Instantiate the StandardScaler object
# X_scaled = scaler.fit_transform(X[["ean_number"]]) # Fit and transform the EAN number column
# X_scaled = pd.DataFrame(X_scaled, columns=["ean_number"]) # Convert to a dataframe with column name

# # Concatenate the date, encoded product name, and scaled EAN number columns
# X = pd.concat([X[["date"]], X_encoded, X_scaled], axis=1) # Concatenate along the column axis
